# Matrix Factorization: Experiments and Evaluation

This notebook implements and compares two classical MF recommendation paradigms:
1. **FunkSVD**
2. **Alternative Least Squares (ALS)**

Both model use low-rank approximation for user-ratings matrix, but differ in optimization strategy: FunkSVD uses stochastic gradient descent (SGD) to iteratively update user- and item- matrix parameters on each epoch, while ALS iteratively fixes one matrix, solves ridge regression for second matrix, then fixes second matrix and solves for the first one until convergence. We use explicit implementations of these algorithm, meaning we try to predict rating of the item. 

In [17]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from src.data.loader import load_all_data
from src.data.splitter import create_temporal_split, SplitConfig, add_random_timestamps, UserTemporalSplitConfig
from src.models import ContentBasedRecommender, ItemItemCFRecommender, FunkSVD, FunkSVDConfig, ALS, ALSConfig, BPRMF, BPRConfig
from src.evaluation import (
    EvaluationPipeline,
    GlobalMeanBaseline,
    UserMeanBaseline,
    ItemMeanBaseline,
    PopularityBaseline,
    print_evaluation_results,
)

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

ImportError: cannot import name 'BPRMF' from 'src.models' (/Users/adrianasluka/Desktop/rec_sys_ucu_2026/notebooks/../src/models/__init__.py)

## 1. Data Loading and Temporal Split

For this task temporal split we use differs from similarity recommendation algorithms. If we split by the year the book was published, matrix factorization inherently make no sense, as all of the validation and test books will face cold start problem. Therefore we create random variable timestamp, that simulates time the book was rated. Then, for each user we order ratings by timestamp and split train/val/test with 0.7/0.15/0.15 ratio, so that older books go in train and newer into validation and test. This creates simulation of temporal data.

In [2]:
# Load data
ratings, books, users = load_all_data('../data/raw')

ratings_ts = add_random_timestamps(ratings, start="2004-08-01", end="2004-09-30", seed=123, ts_col="timestamp")



In [3]:
# Initialize evaluation pipeline
K_VALUES = [10]
pipeline = EvaluationPipeline(k_values=K_VALUES, relevance_threshold=0)

# Store all results
all_results = {}

## 2. Baseline Models

We first establish baselines to compare against.

In [4]:
# Create temporal split
config = UserTemporalSplitConfig(
    train_frac=0.7,
    val_frac=0.15,
    test_frac=0.15,
    min_train=1,
    min_val=1,
    min_test=1,
    min_user_interactions = 10,
    min_item_interactions = 10,
    explicit_only = True
)

train_df, val_df, test_df, split_info = create_temporal_split(ratings_ts, books, "users", config)


--- Step 1: Filtering data ---
Original ratings: 1,149,780
After explicit filter: 433,671 ratings
After year filter (1900-2004): 377,822 ratings
Iteration 1: 71,437 ratings, 6,037 users, 3,244 items
Iteration 2: 46,508 ratings, 2,209 users, 2,237 items
Iteration 3: 42,355 ratings, 1,888 users, 2,066 items
Iteration 4: 41,433 ratings, 1,823 users, 2,027 items
Iteration 5: 41,262 ratings, 1,811 users, 2,020 items
Iteration 6: 41,253 ratings, 1,810 users, 2,020 items
Iteration 7: 41,253 ratings, 1,810 users, 2,020 items

--- Step 2: Temporal split ---
Initial split sizes (by user timestamp):
  Train: 28,144 ratings
  Val: 5,252 ratings
  Test: 7,857 ratings

--- Step 3: Ensuring test users have training history ---

--- Final Split Summary ---
Set             Ratings      Users      Items           Years   Avg Rating
----------------------------------------------------------------------
train            28,144      1,810      2,020       1930-2004         7.96
val               5,252    

In [5]:
# Global Mean Baseline
global_mean = GlobalMeanBaseline()
global_mean.fit(train_df)
all_results['Global Mean'] = pipeline.evaluate_rating_prediction(global_mean, test_df)
print(f"Global Mean → RMSE: {all_results['Global Mean']['rmse']:.4f}, MAE: {all_results['Global Mean']['mae']:.4f}")

# User Mean Baseline
user_mean = UserMeanBaseline()
user_mean.fit(train_df)
all_results['User Mean'] = pipeline.evaluate_rating_prediction(user_mean, test_df)
print(f"User Mean  → RMSE: {all_results['User Mean']['rmse']:.4f}, MAE: {all_results['User Mean']['mae']:.4f}")

# Item Mean Baseline
item_mean = ItemMeanBaseline()
item_mean.fit(train_df)
all_results['Item Mean'] = pipeline.evaluate_rating_prediction(item_mean, test_df)
print(f"Item Mean  → RMSE: {all_results['Item Mean']['rmse']:.4f}, MAE: {all_results['Item Mean']['mae']:.4f}")

Global Mean → RMSE: 1.7306, MAE: 1.3403
User Mean  → RMSE: 1.5629, MAE: 1.1744
Item Mean  → RMSE: 1.7187, MAE: 1.3538


## 3. Alternative Least Squares

In [6]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional


# -----------------------------
# Utilities
# -----------------------------
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1.0 / (1.0 + np.exp(-x))


def make_id_maps(df: pd.DataFrame, user_col="user_id", item_col="isbn") -> Tuple[Dict, Dict, np.ndarray, np.ndarray]:
    """Map raw ids -> contiguous indices [0..n-1]."""
    users = df[user_col].unique()
    items = df[item_col].unique()
    user2idx = {u: k for k, u in enumerate(users)}
    item2idx = {i: k for k, i in enumerate(items)}
    idx2user = users
    idx2item = items
    return user2idx, item2idx, idx2user, idx2item


def build_user_positives(
    df: pd.DataFrame,
    user2idx: Dict,
    item2idx: Dict,
    user_col="user_id",
    item_col="isbn",
    rating_col="rating",
    min_rating_pos: int = 1,
) -> List[np.ndarray]:
    """
    For each user index, store an array of positive item indices.
    Positives: rating >= min_rating_pos (default: >0, i.e., explicit positives in Book-Crossing).
    """
    n_users = len(user2idx)
    pos_lists: List[List[int]] = [[] for _ in range(n_users)]
    sub = df[df[rating_col] >= min_rating_pos][[user_col, item_col]].drop_duplicates()

    for u, i in sub.itertuples(index=False):
        if u in user2idx and i in item2idx:
            pos_lists[user2idx[u]].append(item2idx[i])

    # convert to numpy arrays
    return [np.asarray(lst, dtype=np.int32) for lst in pos_lists]


def build_seen_sets(
    df: pd.DataFrame,
    user2idx: Dict,
    item2idx: Dict,
    user_col="userId",
    item_col="itemId",
) -> List[set]:
    """Items seen in train (for filtering during recommend)."""
    n_users = len(user2idx)
    seen = [set() for _ in range(n_users)]
    sub = df[[user_col, item_col]].drop_duplicates()
    for u, i in sub.itertuples(index=False):
        if u in user2idx and i in item2idx:
            seen[user2idx[u]].add(item2idx[i])
    return seen


In [7]:


# -----------------------------
# Negative samplers
# -----------------------------
class NegativeSampler:
    def sample(self, size: int) -> np.ndarray:
        raise NotImplementedError


class UniformNegativeSampler(NegativeSampler):
    def __init__(self, n_items: int, rng: np.random.Generator):
        self.n_items = n_items
        self.rng = rng

    def sample(self, size: int) -> np.ndarray:
        return self.rng.integers(0, self.n_items, size=size, dtype=np.int32)


class PopularityNegativeSampler(NegativeSampler):
    def __init__(self, item_pop_counts: np.ndarray, rng: np.random.Generator, alpha: float = 1.0):
        """
        item_pop_counts: shape (n_items,)
        alpha: >1 emphasizes head items more; <1 flattens.
        """
        self.rng = rng
        p = np.asarray(item_pop_counts, dtype=np.float64) ** alpha
        p = p / p.sum()
        self.p = p

    def sample(self, size: int) -> np.ndarray:
        return self.rng.choice(len(self.p), size=size, replace=True, p=self.p).astype(np.int32)


class MixedNegativeSampler(NegativeSampler):
    def __init__(self, sampler_a: NegativeSampler, sampler_b: NegativeSampler, p_a: float, rng: np.random.Generator):
        self.a = sampler_a
        self.b = sampler_b
        self.p_a = float(p_a)
        self.rng = rng

    def sample(self, size: int) -> np.ndarray:
        mask = self.rng.random(size) < self.p_a
        out = np.empty(size, dtype=np.int32)
        n_a = int(mask.sum())
        out[mask] = self.a.sample(n_a)
        out[~mask] = self.b.sample(size - n_a)
        return out


In [11]:
# -----------------------------
# Example: putting it together
# -----------------------------
def train_bpr_on_bookcrossing(
    train_df: pd.DataFrame,
    user_col="user_id",
    item_col="isbn",
    rating_col="rating",
    timestamp_col="timestamp",
    min_rating_pos: int = 1,   # treat rating>0 as positive
    neg_strategy: str = "mixed",  # "uniform" | "pop" | "mixed"
    pop_alpha: float = 1.0,
    mixed_p_uniform: float = 0.5,
    cfg: Optional[BPRConfig] = None,
) -> Tuple[BPRMF, Dict, Dict, np.ndarray, np.ndarray, List[np.ndarray], List[set]]:
    """
    Returns:
      model, user2idx, item2idx, idx2user, idx2item, user_pos, user_seen
    """
    cfg = cfg or BPRConfig()

    # Build id maps from TRAIN only (important: avoid leakage of unseen test items)
    user2idx, item2idx, idx2user, idx2item = make_id_maps(train_df, user_col, item_col)

    n_users = len(user2idx)
    n_items = len(item2idx)

    # positives and seen
    user_pos = build_user_positives(train_df, user2idx, item2idx, user_col, item_col, rating_col, min_rating_pos)
    user_seen = build_seen_sets(train_df, user2idx, item2idx, user_col, item_col)

    # item popularity (by #users who interacted) for pop negatives
    # (use all interactions or only positives; pick one and be consistent)
    tmp = train_df[[user_col, item_col]].drop_duplicates()
    pop_counts = tmp[item_col].map(item2idx).value_counts().reindex(range(n_items), fill_value=0).values

    rng = np.random.default_rng(cfg.seed)
    uniform = UniformNegativeSampler(n_items, rng)
    pop = PopularityNegativeSampler(pop_counts, rng, alpha=pop_alpha)

    if neg_strategy == "uniform":
        sampler = uniform
    elif neg_strategy == "pop":
        sampler = pop
    elif neg_strategy == "mixed":
        sampler = MixedNegativeSampler(uniform, pop, p_a=mixed_p_uniform, rng=rng)
    else:
        raise ValueError("neg_strategy must be one of: 'uniform', 'pop', 'mixed'")

    model = BPRMF(n_users, n_items, cfg)
    model.fit(user_pos=user_pos, neg_sampler=sampler, verbose=True)

    return model, user2idx, item2idx, idx2user, idx2item, user_pos, user_seen


In [10]:
# -----------------------------
# BPR (matrix factorization) implementation
# -----------------------------
@dataclass
class BPRConfig:
    n_factors: int = 64
    lr: float = 0.05
    reg: float = 1e-4           # L2 on embeddings (and item bias if used)
    n_epochs: int = 20
    batch_size: int = 2048
    n_samples_per_epoch: int = 200_000  # number of (u,i,j) triples per epoch
    seed: int = 42
    use_item_bias: bool = True


class BPRMF:
    """
    BPR-OPT with MF scoring: s(u,i) = p_u^T q_i + b_i
    Optimizes: -log sigma(s(u,i) - s(u,j)) + L2 regularization
    """

    def __init__(self, n_users: int, n_items: int, cfg: BPRConfig):
        self.n_users = n_users
        self.n_items = n_items
        self.cfg = cfg
        self.rng = np.random.default_rng(cfg.seed)

        # init embeddings
        scale = 0.1
        self.P = (self.rng.normal(0, scale, size=(n_users, cfg.n_factors))).astype(np.float32)
        self.Q = (self.rng.normal(0, scale, size=(n_items, cfg.n_factors))).astype(np.float32)
        self.b = np.zeros(n_items, dtype=np.float32) if cfg.use_item_bias else None

    def score(self, u_idx: np.ndarray, i_idx: np.ndarray) -> np.ndarray:
        s = np.sum(self.P[u_idx] * self.Q[i_idx], axis=1)
        if self.b is not None:
            s = s + self.b[i_idx]
        return s

    def fit(
        self,
        user_pos: List[np.ndarray],
        neg_sampler: NegativeSampler,
        verbose: bool = True,
    ) -> List[float]:
        """
        user_pos: list length n_users, each an array of positive item indices
        neg_sampler: defines how we sample j from items
        """
        cfg = self.cfg
        losses = []

        # users with at least 1 positive
        eligible_users = np.array([u for u in range(self.n_users) if len(user_pos[u]) > 0], dtype=np.int32)
        if len(eligible_users) == 0:
            raise ValueError("No eligible users with positives. Check your min_rating_pos or data filtering.")

        for epoch in range(cfg.n_epochs):
            epoch_loss = 0.0
            n_done = 0

            # iterate in mini-batches over randomly sampled triples
            n_total = cfg.n_samples_per_epoch
            while n_done < n_total:
                bs = min(cfg.batch_size, n_total - n_done)

                # sample users
                u = self.rng.choice(eligible_users, size=bs, replace=True)

                # sample a positive i for each user u
                i = np.empty(bs, dtype=np.int32)
                for k, uu in enumerate(u):
                    pos_items = user_pos[uu]
                    i[k] = pos_items[self.rng.integers(0, len(pos_items))]

                # sample negatives j (and resample if accidentally positive for that user)
                j = neg_sampler.sample(bs)
                #print('negatives', j)

                # reject positives (simple loop; ok for typical data; optimize if needed)
                for k, uu in enumerate(u):
                    pos_set = user_pos[uu]
                    #print('positives', pos_set)
                    # fast containment if we convert to set per user; but array is ok if small.
                    # We'll do a small while with np.any on array.
                    while np.any(pos_set == j[k]):
                        j[k] = int(neg_sampler.sample(1)[0])

                # scores
                x_ui = self.score(u, i)
                x_uj = self.score(u, j)
                x_uij = x_ui - x_uj

                # BPR loss = -log sigmoid(x_uij)
                # gradient factor = sigmoid(-x_uij) = 1 - sigmoid(x_uij)
                s = sigmoid(x_uij).astype(np.float32)
                g = (1.0 - s)  # = sigmoid(-x_uij)

                # Fetch embeddings
                Pu = self.P[u]          # (bs, f)
                Qi = self.Q[i]
                Qj = self.Q[j]

                # Gradients (vectorized)
                # d/dPu:  g[:,None]*(Qi - Qj) - reg*Pu
                # d/dQi:  g[:,None]*Pu       - reg*Qi
                # d/dQj: -g[:,None]*Pu       - reg*Qj
                reg = cfg.reg
                lr = cfg.lr

                dPu = (g[:, None] * (Qi - Qj)) - reg * Pu
                dQi = (g[:, None] * Pu) - reg * Qi
                dQj = (-g[:, None] * Pu) - reg * Qj

                # Apply updates (SGD)
                # Note: repeated indices exist; numpy fancy assignment won't accumulate.
                # We do per-row updates via np.add.at for correctness.
                np.add.at(self.P, u, lr * dPu)
                np.add.at(self.Q, i, lr * dQi)
                np.add.at(self.Q, j, lr * dQj)

                if self.b is not None:
                    # d/db_i:  g - reg*b_i ; d/db_j: -g - reg*b_j
                    db_i = g - reg * self.b[i]
                    db_j = -g - reg * self.b[j]
                    np.add.at(self.b, i, lr * db_i)
                    np.add.at(self.b, j, lr * db_j)

                # batch loss value
                batch_loss = -np.log(np.clip(s, 1e-8, 1.0)).mean()
                epoch_loss += batch_loss * bs
                n_done += bs

            epoch_loss /= n_total
            losses.append(epoch_loss)
            if verbose:
                print(f"Epoch {epoch+1:02d}/{cfg.n_epochs} | train_bpr_loss={epoch_loss:.5f}")

        return losses

    def recommend(
        self,
        u_idx: int,
        K: int,
        seen_items: Optional[set] = None,
        candidates: Optional[np.ndarray] = None,
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Return (item_indices, scores) for top-K items.
        If candidates is None: scores all items (can be slow for huge catalogs).
        Filters seen_items if provided.
        """
        if candidates is None:
            cand = np.arange(self.n_items, dtype=np.int32)
        else:
            cand = candidates.astype(np.int32)

        if seen_items:
            mask = np.array([i not in seen_items for i in cand], dtype=bool)
            cand = cand[mask]

        # score
        u_arr = np.full(len(cand), u_idx, dtype=np.int32)
        s = self.score(u_arr, cand)

        if len(cand) <= K:
            order = np.argsort(-s)
            return cand[order], s[order]

        # partial top-k
        topk_idx = np.argpartition(-s, K)[:K]
        topk_sorted = topk_idx[np.argsort(-s[topk_idx])]
        return cand[topk_sorted], s[topk_sorted]


In [16]:
cfg = BPRConfig(
    n_factors=64,
    lr=0.05,
    reg=1e-4,
    n_epochs=25,
    batch_size=2048,
    n_samples_per_epoch=200_000,
    seed=42,
    use_item_bias=True,
)

model, user2idx, item2idx, idx2user, idx2item, user_pos, user_seen = train_bpr_on_bookcrossing(
    train_df,
    min_rating_pos=1,          # positives = rating>0
    neg_strategy="uniform",      # try: "uniform", "pop", "mixed"
    pop_alpha=1.0,
    mixed_p_uniform=0.5,       # prob of uniform vs popularity in mixed sampler
    cfg=cfg,
)

metrics = evaluate_bpr(model, test_df, user2idx, item2idx, user_seen, min_rating_pos=1, K=10)
print(metrics)


Epoch 01/25 | train_bpr_loss=0.53562
Epoch 02/25 | train_bpr_loss=0.33023
Epoch 03/25 | train_bpr_loss=0.17563
Epoch 04/25 | train_bpr_loss=0.10264
Epoch 05/25 | train_bpr_loss=0.06858
Epoch 06/25 | train_bpr_loss=0.04982
Epoch 07/25 | train_bpr_loss=0.03923
Epoch 08/25 | train_bpr_loss=0.03172
Epoch 09/25 | train_bpr_loss=0.02722
Epoch 10/25 | train_bpr_loss=0.02356
Epoch 11/25 | train_bpr_loss=0.02061
Epoch 12/25 | train_bpr_loss=0.01860
Epoch 13/25 | train_bpr_loss=0.01703
Epoch 14/25 | train_bpr_loss=0.01520
Epoch 15/25 | train_bpr_loss=0.01395
Epoch 16/25 | train_bpr_loss=0.01314
Epoch 17/25 | train_bpr_loss=0.01208
Epoch 18/25 | train_bpr_loss=0.01141
Epoch 19/25 | train_bpr_loss=0.01055
Epoch 20/25 | train_bpr_loss=0.01008
Epoch 21/25 | train_bpr_loss=0.00960
Epoch 22/25 | train_bpr_loss=0.00905
Epoch 23/25 | train_bpr_loss=0.00875
Epoch 24/25 | train_bpr_loss=0.00833
Epoch 25/25 | train_bpr_loss=0.00768
uu 1
recs ['0316155306', '0316693235', '0399150897', '0316603570', '0515122

In [15]:
# Evaluate FunkSVD - Ranking
print("Evaluating ranking (this may take a few minutes)...")
t0 = time.time()
model_ranking = pipeline.evaluate_ranking(model, test_df, train_df, n_recommendations=10, user2idx = user2idx, idx2item = idx2item, item2idx = item2idx)
print(f"  NDCG@10:     {model_ranking.get('ndcg@10', 0):.4f}")
print(f"  Precision@10: {model_ranking.get('precision@10', 0):.4f}")
print(f"  Hit Rate@10:  {model_ranking.get('hit_rate@10', 0):.4f}")
print(f"  Users eval'd: {model_ranking.get('users_evaluated', 0)}")
print(f"  Time: {time.time()-t0:.1f}s")

Evaluating ranking (this may take a few minutes)...
1810
  NDCG@10:     0.0388
  Precision@10: 0.0194
  Hit Rate@10:  0.1707
  Users eval'd: 1810
  Time: 1.9s


In [17]:

metrics = evaluate_bpr(model, test_df, user2idx, item2idx, user_seen, min_rating_pos=1, K=10)
print(metrics)

uu 1
recs ['0316155306', '0316693235', '0399150897', '0316603570', '0515122734', '0440225701', '0345435168', '0142001740', '0425182908', '0525947299']
gtu ['0316603570', '0684835983', '0743206045']
uu 2
recs ['0515136530', '0515133973', '0515131229', '0515122734', '0553280368', '0553295977', '0373484410', '0515132020', '0515116750', '051513628X']
gtu ['0515136530', '0451203771', '0515134384']
uu 12
recs ['0452282152', '0345337662', '059035342X', '0439064872', '043935806X', '0375727345', '0618002235', '0345313860', '0062502182', '0743225406']
gtu ['0066238501', '0439064872', '0380789035']
uu 18
recs ['0440206154', '0440220602', '0452282152', '0525947647', '0679746048', '155874262X', '0425143325', '0451184963', '0385484518', '0451180232']
gtu ['0060915544', '0451208765', '155874262X']
uu 21
recs ['0439136350', '0836218663', '0060502258', '0439139597', '0812550706', '0385484518', '0836218620', '034538475X', '1573227889', '1573229725']
gtu ['0439139597', '0060248025', '0142000663', '059035

In [13]:


# -----------------------------
# Minimal ranking evaluation helpers (binary relevance)
# -----------------------------
def precision_at_k(recs: List[int], gt: set, k: int) -> float:
    if k == 0:
        return 0.0
    return len(set(recs[:k]) & gt) / k


def hitrate_at_k(recs: List[int], gt: set, k: int) -> float:
    return 1.0 if len(set(recs[:k]) & gt) > 0 else 0.0


def ndcg_at_k(recs: List[int], gt: set, k: int) -> float:
    # binary relevance
    dcg = 0.0
    for rank, item in enumerate(recs[:k], start=1):
        if item in gt:
            dcg += 1.0 / np.log2(rank + 1)
    # ideal DCG
    ideal_hits = min(len(gt), k)
    if ideal_hits == 0:
        return 0.0
    idcg = sum(1.0 / np.log2(r + 1) for r in range(1, ideal_hits + 1))
    return dcg / idcg


def evaluate_bpr(
    model: BPRMF,
    test_df: pd.DataFrame,
    user2idx: Dict,
    item2idx: Dict,
    user_seen: List[set],
    user_col="user_id",
    item_col="isbn",
    rating_col="rating",
    min_rating_pos: int = 1,  # relevance in test: rating>0 by default
    K: int = 10,
    max_users: Optional[int] = None,
) -> Dict[str, float]:
    """
    Evaluates on users present in user2idx and with at least 1 relevant test item.
    """
    # build gt per user (mapped to indices, only items known in train map)
    df = test_df[test_df[rating_col] >= min_rating_pos][[user_col, item_col]].drop_duplicates()
    gt: Dict[int, set] = {}
    for u, i in df.itertuples(index=False):
        if u in user2idx and i in item2idx:
            uu = user2idx[u]
            ii = item2idx[i]
            gt.setdefault(uu, set()).add(ii)

    users = list(gt.keys())
    if max_users is not None:
        users = users[:max_users]

    precs, hits, ndcgs = [], [], []
    for uu in users:
        items, _scores = model.recommend(uu, K=K, seen_items=user_seen[uu])
        recs = items.tolist()
        gtu = gt[uu]
        if hitrate_at_k(recs, gtu, K) == 1:
            print('uu', uu)
            print('recs', [idx2item[i] for i in recs])
            print('gtu', [idx2item[i] for i in gtu])
        precs.append(precision_at_k(recs, gtu, K))
        hits.append(hitrate_at_k(recs, gtu, K))
        ndcgs.append(ndcg_at_k(recs, gtu, K))

    return {
        "users_eval": float(len(users)),
        f"precision@{K}": float(np.mean(precs)) if precs else 0.0,
        f"hitrate@{K}": float(np.mean(hits)) if hits else 0.0,
        f"ndcg@{K}": float(np.mean(ndcgs)) if ndcgs else 0.0,
    }


"""
HOW TO USE (example):

# train_df, test_df are pandas dataframes with columns: userId, itemId, rating, timestamp

cfg = BPRConfig(
    n_factors=64,
    lr=0.05,
    reg=1e-4,
    n_epochs=20,
    batch_size=2048,
    n_samples_per_epoch=200_000,
    seed=42,
    use_item_bias=True,
)

model, user2idx, item2idx, idx2user, idx2item, user_pos, user_seen = train_bpr_on_bookcrossing(
    train_df,
    min_rating_pos=1,          # positives = rating>0
    neg_strategy="mixed",      # try: "uniform", "pop", "mixed"
    pop_alpha=1.0,
    mixed_p_uniform=0.5,       # prob of uniform vs popularity in mixed sampler
    cfg=cfg,
)

metrics = evaluate_bpr(model, test_df, user2idx, item2idx, user_seen, min_rating_pos=1, K=10)
print(metrics)
"""

'\nHOW TO USE (example):\n\n# train_df, test_df are pandas dataframes with columns: userId, itemId, rating, timestamp\n\ncfg = BPRConfig(\n    n_factors=64,\n    lr=0.05,\n    reg=1e-4,\n    n_epochs=20,\n    batch_size=2048,\n    n_samples_per_epoch=200_000,\n    seed=42,\n    use_item_bias=True,\n)\n\nmodel, user2idx, item2idx, idx2user, idx2item, user_pos, user_seen = train_bpr_on_bookcrossing(\n    train_df,\n    min_rating_pos=1,          # positives = rating>0\n    neg_strategy="mixed",      # try: "uniform", "pop", "mixed"\n    pop_alpha=1.0,\n    mixed_p_uniform=0.5,       # prob of uniform vs popularity in mixed sampler\n    cfg=cfg,\n)\n\nmetrics = evaluate_bpr(model, test_df, user2idx, item2idx, user_seen, min_rating_pos=1, K=10)\nprint(metrics)\n'

In [18]:
idx2item

array(['0553571656', '0679736042', '0345413903', ..., '0380762587',
       '0061096083', '0060256656'], dtype=object)